In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score

In [4]:
array_of_labeled_csv=[#'Bioderma_labeled.csv',
                      #'erave_labeled.csv',
                      #'neutrogena_labeled.csv',
                      #'labeled_mayb_foundation.csv',
                      #'labeled_mayb_conc.csv',
                      #'labeled_hp_envy.csv',
                      
    #'labeled_Macbook m1.csv',
     #                 'labeled_lenovo x1.csv',
      #                'labeled_lenovo e14.csv',
                       'galaxy7_labeled.csv'
                     ]
for product in range(len(array_of_labeled_csv)):
    data = pd.read_csv(array_of_labeled_csv[product])
    data.drop(['Positive','Negative','Neutral','Compound'],axis=1,inplace=True)
    data.drop(data[(data['Sentiment'] =='Neutral')].index, inplace=True)
    df=data
    df['Sentiment'] = df['Sentiment'].replace(['Positive'], 1)
    df['Sentiment'] = df['Sentiment'].replace(['Negative'], 0)
    train, test = train_test_split(df, test_size=0.2, random_state=1)
    X_train = train['Review'].values
    X_test = test['Review'].values
    y_train = train['Sentiment']
    y_test = test['Sentiment']
    def tokenize(text): 
        tknzr = TweetTokenizer()
        return tknzr.tokenize(text)

    def stem(doc):
        return (stemmer.stem(w) for w in analyzer(doc))

    en_stopwords = set(stopwords.words("english")) 

    vectorizer = CountVectorizer(
        analyzer = 'word',
        tokenizer = tokenize,
        lowercase = True,
        ngram_range=(1, 1),
        stop_words = en_stopwords)
    kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    np.random.seed(1)

    pipeline_svm = make_pipeline(vectorizer, 
                                SVC(probability=True, kernel="linear", class_weight="balanced"))

    grid_svm = GridSearchCV(pipeline_svm,
                        param_grid = {'svc__C': [0.01, 0.1, 1]}, 
                        cv = kfolds,
                        scoring="roc_auc",
                        verbose=1,   
                        n_jobs=-1) 

    grid_svm.fit(X_train, y_train)
    #print(grid_svm.score(X_test, y_test)*100)
    productname=array_of_labeled_csv[product]
    accuaracy=grid_svm.score(X_test, y_test)*100
    
    #print("Accuracy : ", accuaracy,"%")
    f = open("Accuracies.txt", "a")
    f.write("\n"+productname+ " " + str(accuaracy)+"\n")
    f.close()

Fitting 5 folds for each of 3 candidates, totalling 15 fits
